In [1]:
import os
import pandas as pd
from pandas import read_csv, read_excel, DataFrame
import sae
import gurobipy as gp
from gurobipy import GRB
import random

import sys
sys.path.append('../')
import saedfsc

In [2]:
saedfsc.suppliers

,Name,Location,Rating,SetUpCost
0,Supplier1,Location1,1.178742,4303
1,Supplier2,Location2,2.930649,2174
2,Supplier3,Location1,4.821298,2699
3,Supplier4,Location4,2.837146,2724
4,Supplier5,Location2,3.659921,4715
5,Supplier6,Location2,1.560158,2056
6,Supplier7,Location3,1.165378,3938


In [3]:
partOptions = saedfsc.getPartOptionsWithSuppliers()

In [4]:
# Print Wings Dataframe For Reference
partOptions['wings']

,Unnamed: 0_x,Material,Code,q,E,cost_per_kilogram,q (Ordering materials),Unnamed: 0_y,length,height,width,angle of attack,length*heigth*width (Ordering Wings),Suppliers
909,4,Glass-filled nylon (35%),GFN,1600,14900000000,2.40,1600,45,0.30,0.200,0.05,0.157,0.003000,"[Supplier6, Supplier3]"
396,1,Nylon 6/6,N66,1150,2400000000,3.30,1150,180,0.55,0.200,2.54,0.785,0.279400,"[Supplier3, Supplier4]"
885,4,Glass-filled nylon (35%),GFN,1600,14900000000,2.40,1600,21,0.30,0.050,0.05,0.785,0.000750,"[Supplier4, Supplier1, Supplier7]"
1255,5,S-glass epoxy (45%),SGE,1800,39500000000,3.76,1800,175,0.55,0.125,3.00,0.785,0.206250,[Supplier6]
1354,6,Glass-filled epoxy (35%),GFE,1900,25000000000,3.86,1900,58,0.55,0.200,0.05,0.785,0.005500,"[Supplier7, Supplier1, Supplier3]"
2591,11,"Cast Iron, grade 20",CI20,7150,100000000000,1.94,7150,215,0.55,0.550,3.00,0.785,0.907500,"[Supplier3, Supplier1, Supplier5]"
1642,7,Glass-filled polyester (35%),GFP,2000,15700000000,2.40,2000,130,0.05,0.550,2.54,0.628,0.069850,"[Supplier4, Supplier3]"
28,0,Polyethylene (HDPE),HDPE,900,900000000,0.85,900,28,0.55,0.050,0.05,0.000,0.001375,"[Supplier7, Supplier5]"
2532,11,"Cast Iron, grade 20",CI20,7150,100000000000,1.94,7150,156,0.30,0.200,2.54,0.314,0.152400,"[Supplier5, Supplier3, Supplier1]"
1858,8,Aluminum 2045-T4,AT4,2700,73000000000,80.00,2700,130,0.05,0.550,2.54,0.628,0.069850,"[Supplier6, Supplier2]"


In [5]:
# Print Engines Dataframe For Reference
partOptions['engine']

,Unnamed: 0,ID,Name,Length,Width,Height,Mass,Power,Torque,Cost,Mass*Power (ordering motors),Suppliers
0,0,M1,GX200,0.321,0.376,0.346,16.1,4100,12.40,650,66010,"[Supplier1, Supplier7, Supplier6]"
1,1,M3,GX270,0.380,0.429,0.422,25.0,6300,19.10,880,157500,"[Supplier7, Supplier4]"
2,2,M2,GX240,0.380,0.429,0.422,27.5,5900,18.30,870,162250,"[Supplier2, Supplier7]"
3,3,M20,iGX270,0.384,0.468,0.303,26.2,6300,19.10,880,165060,"[Supplier1, Supplier7]"
4,4,M21,iGX 240 (with fuel tank?),0.355,0.462,0.422,30.3,5900,18.30,870,178770,[Supplier2]
5,5,M5,GXV340,0.433,0.382,0.406,32.3,6600,21.60,915,213180,"[Supplier6, Supplier3]"
6,6,M15,GD320,0.445,0.460,0.465,48.0,5200,15.70,900,249600,"[Supplier3, Supplier6]"
7,7,M23,iGX340 (low profile),0.405,0.484,0.313,31.4,8000,26.40,915,251200,"[Supplier2, Supplier1]"
8,8,M4,GX340,0.407,0.485,0.449,31.5,8000,26.40,915,252000,"[Supplier2, Supplier5, Supplier7]"
9,9,M7,GXV390,0.433,0.382,0.406,33.3,7600,24.20,1020,253080,"[Supplier7, Supplier5]"


In [6]:
# Print Cabins Dataframe For Reference
partOptions['cabin']

,Unnamed: 0_x,Material,Code,q,E,cost_per_kilogram,q (Ordering materials),Unnamed: 0_y,length,height,width,thickness,length*heigth*width (Ordering Cabins),Suppliers
995,5,S-glass epoxy (45%),SGE,1800,39500000000,3.76,1800,35,1.91,0.88,0.50,0.012,0.840400,"[Supplier6, Supplier2, Supplier4]"
464,2,Epoxy,EPY,1250,3350000000,2.76,1250,80,2.39,1.17,0.50,0.012,1.398150,[Supplier1]
107,0,Polyethylene (HDPE),HDPE,900,900000000,0.85,900,107,1.91,1.17,0.79,0.012,1.765413,"[Supplier3, Supplier2]"
1677,8,Aluminum 2045-T4,AT4,2700,73000000000,80.00,2700,141,2.39,0.88,1.26,0.004,2.650032,[Supplier3]
1783,9,MgO,MGO,3600,205000000000,134.00,3600,55,1.52,0.88,0.79,0.008,1.056704,[Supplier2]
553,2,Epoxy,EPY,1250,3350000000,2.76,1250,169,1.52,1.17,2.00,0.008,3.556800,"[Supplier4, Supplier5, Supplier2]"
1326,6,Glass-filled epoxy (35%),GFE,1900,25000000000,3.86,1900,174,2.39,0.88,2.00,0.004,4.206400,"[Supplier6, Supplier7]"
177,0,Polyethylene (HDPE),HDPE,900,900000000,0.85,900,177,3.00,1.17,1.26,0.004,4.422600,"[Supplier3, Supplier1, Supplier5]"
1786,9,MgO,MGO,3600,205000000000,134.00,3600,58,1.91,1.17,0.50,0.008,1.117350,"[Supplier1, Supplier7, Supplier3]"
521,2,Epoxy,EPY,1250,3350000000,2.76,1250,137,3.00,0.66,1.26,0.012,2.494800,[Supplier3]


In [7]:
# Print Tires Dataframe For Reference
partOptions['tires']

,Unnamed: 0_x,Pressure Rating,pressure,pressure (Ordering Pressure Ratings),Unnamed: 0_y,ID,radius,mass,cost,radius (Ordering tires),Suppliers
0,0,Basic,0.758,0.758,0,T1,0.22860,3.636,300,0.22860,"[Supplier3, Supplier4, Supplier2]"
1,0,Basic,0.758,0.758,1,T2,0.22987,4.091,302,0.22987,[Supplier5]
2,0,Basic,0.758,0.758,2,T3,0.23241,4.545,306,0.23241,[Supplier4]
3,0,Basic,0.758,0.758,3,T4,0.24638,4.545,324,0.24638,"[Supplier5, Supplier4, Supplier2]"
4,0,Basic,0.758,0.758,4,T5,0.24765,5.000,326,0.24765,"[Supplier6, Supplier4, Supplier7]"
5,0,Basic,0.758,0.758,5,T8,0.26162,5.455,344,0.26162,[Supplier5]
6,0,Basic,0.758,0.758,6,T6,0.26670,5.000,352,0.26670,[Supplier1]
7,1,Low,0.834,0.834,0,T1,0.22860,3.636,300,0.22860,[Supplier3]
8,1,Low,0.834,0.834,1,T2,0.22987,4.091,302,0.22987,"[Supplier7, Supplier4]"
9,1,Low,0.834,0.834,2,T3,0.23241,4.545,306,0.23241,[Supplier3]


In [8]:
# Print Brakes Dataframe For Reference
partOptions['brakes']

,Unnamed: 0,brakeID,qbrk,lbrk,hbrk,wbrk,tbrk,rbrk,qbrk*lbrk*hbrk*wbrk*rbrk (Ordering brakes),Suppliers
0,0,B15,0.50,0.1040,0.0325,0.0160,0.320,0.030,8.110000e-07,"[Supplier4, Supplier2, Supplier5]"
1,1,B1,0.40,0.1000,0.0425,0.0200,0.350,0.025,8.500000e-07,[Supplier5]
2,2,B16,0.50,0.1135,0.0400,0.0160,0.350,0.025,9.080000e-07,"[Supplier1, Supplier3]"
3,3,B4,0.50,0.1000,0.0380,0.0160,0.350,0.030,9.120000e-07,"[Supplier2, Supplier7]"
4,4,B6,0.50,0.1040,0.0375,0.0160,0.355,0.030,9.360000e-07,"[Supplier4, Supplier2, Supplier7]"
5,5,B2,0.40,0.1400,0.0490,0.0175,0.350,0.025,1.200000e-06,[Supplier3]
6,6,B18,0.52,0.1000,0.0425,0.0200,0.350,0.028,1.240000e-06,"[Supplier1, Supplier7, Supplier5]"
7,7,B7,0.50,0.1100,0.0520,0.0140,0.350,0.032,1.280000e-06,"[Supplier6, Supplier3]"
8,8,B5,0.50,0.1000,0.0425,0.0200,0.355,0.032,1.360000e-06,"[Supplier3, Supplier1]"
9,9,B8,0.50,0.1320,0.0420,0.0180,0.355,0.028,1.400000e-06,[Supplier4]


In [9]:
# Print Suspension Dataframe For Reference
partOptions['suspension']

,Unnamed: 0,id,krsp,crsp,mrsp,kfsp,cfsp,mfsp,krsp*crsp*mrsp*kfsp*cfsp*mfsp (Ordering suspensions),Suppliers
0,0,S1,4000,600,0.3,4000,600,0.3,5.180000e+11,"[Supplier2, Supplier5, Supplier7]"
1,1,S2,21000,2000,0.3,21000,2000,0.3,1.590000e+14,[Supplier5]
2,2,S8,24000,2250,0.3,24000,2250,0.3,2.620000e+14,"[Supplier1, Supplier7]"
3,3,S9,26000,2300,0.3,26000,2300,0.3,3.220000e+14,"[Supplier1, Supplier4]"
4,4,S10,28000,2500,0.3,28000,2500,0.3,4.410000e+14,"[Supplier5, Supplier7]"


In [10]:
# Print Impact Attenuator Dataframe For Reference
partOptions['impactattenuator']

,Unnamed: 0_x,Material,Code,q,E,cost_per_kilogram,q (Ordering materials),Unnamed: 0_y,length,height,width,length*heigth*width (Ordering Impact Attenuator),Suppliers
73,1,Nylon 6/6,N66,1150,2400000000,3.30,1150,9,0.20,0.5,0.5,0.0500,"[Supplier4, Supplier7]"
550,8,Aluminum 2045-T4,AT4,2700,73000000000,80.00,2700,38,0.50,0.5,0.5,0.1250,[Supplier5]
117,1,Nylon 6/6,N66,1150,2400000000,3.30,1150,53,0.65,0.5,0.5,0.1625,[Supplier6]
372,5,S-glass epoxy (45%),SGE,1800,39500000000,3.76,1800,52,0.65,0.5,0.5,0.1625,[Supplier2]
216,3,Kevlar epoxy (53%),KE,1350,63600000000,4.20,1350,24,0.35,0.5,0.5,0.0875,"[Supplier5, Supplier3, Supplier1]"
8,0,Polyethylene (HDPE),HDPE,900,900000000,0.85,900,8,0.20,0.5,0.5,0.0500,[Supplier4]
233,3,Kevlar epoxy (53%),KE,1350,63600000000,4.20,1350,41,0.50,0.5,0.5,0.1250,"[Supplier6, Supplier4, Supplier2]"
331,5,S-glass epoxy (45%),SGE,1800,39500000000,3.76,1800,11,0.20,0.5,0.5,0.0500,[Supplier7]
743,11,"Cast Iron, grade 20",CI20,7150,100000000000,1.94,7150,39,0.50,0.5,0.5,0.1250,"[Supplier4, Supplier3]"
368,5,S-glass epoxy (45%),SGE,1800,39500000000,3.76,1800,48,0.65,0.5,0.5,0.1625,"[Supplier1, Supplier7]"


In [11]:
# How to acces row indices and column values

print(partOptions['impactattenuator'].index)

print()

for i in range(len(partOptions['impactattenuator'].index)):
    print(partOptions['impactattenuator'].index[i], partOptions['impactattenuator'].iloc[i]['Material'])

Int64Index([ 73, 550, 117, 372, 216,   8, 233, 331, 743, 368, 737,  94, 636,
            522, 650, 318, 804, 392, 363, 787],
           dtype='int64')

73 Nylon 6/6
550 Aluminum 2045-T4
117 Nylon 6/6
372 S-glass epoxy (45%)
216 Kevlar epoxy (53%)
8 Polyethylene (HDPE)
233 Kevlar epoxy (53%)
331 S-glass epoxy (45%)
743 Cast Iron, grade 20
368 S-glass epoxy (45%)
737 Cast Iron, grade 20
94 Nylon 6/6
636 MgO
522 Aluminum 2045-T4
650 Alumina
318 Glass-filled nylon (35%)
804 Steel, AISI 1045
392 Glass-filled epoxy (35%)
363 S-glass epoxy (45%)
787 Steel, AISI 1045


In [12]:
# Small Example

# Initialize
column_titles = ['Supplier', 'Broader Part', 'Specific Part: Row Index', 'Price Level', 'MinQty', 'Discount']

qds_df_example = pd.DataFrame(columns = column_titles)

qds_df_example

# Populate qds dataframe

row_to_add = ['Supplier 4', 'Impact Attenuator', partOptions['impactattenuator'].index[0], 1, 10, 0.2]

second_row_to_add = ['Supplier 7', 'Wings', partOptions['wings'].index[5], 2, 25, 0.3]

third_row_to_add = ['Supplier 1', 'Engine', partOptions['engine'].index[0], 3, 40, 0.5]

qds_df_example.loc[len(qds_df_example.index)] = row_to_add

qds_df_example.loc[len(qds_df_example.index)] = second_row_to_add

qds_df_example.loc[len(qds_df_example.index)] = third_row_to_add

qds_df_example

,Supplier,Broader Part,Specific Part: Row Index,Price Level,MinQty,Discount
0,Supplier 4,Impact Attenuator,73,1,10,0.2
1,Supplier 7,Wings,2591,2,25,0.3
2,Supplier 1,Engine,0,3,40,0.5


In [13]:
# Function Input: suppliers

supplier_list = list(saedfsc.suppliers.iloc[:]['Name'])

print('Supplier List:', supplier_list)
print()

# Function Input: broader_parts

parts_list = ['wings', 'engine', 'cabin', 'tires', 'brakes', 'suspension', 'impactattenuator']

print('Part List:', parts_list)

Supplier List: ['Supplier1', 'Supplier2', 'Supplier3', 'Supplier4', 'Supplier5', 'Supplier6', 'Supplier7']

Part List: ['wings', 'engine', 'cabin', 'tires', 'brakes', 'suspension', 'impactattenuator']


In [14]:
# Full Function

def make_quantity_discount_schedule(Suppliers, part_options, Broad_parts, random_seed):
    random.seed(random_seed)
    column_titles = ['Supplier', 'Broader Part', 'Specific Part: Row Index', 'Price Level', 'MinQty', 'Discount']
    qds_df = pd.DataFrame(columns = column_titles)
    
    for supplier in Suppliers:
        for broad_part in Broad_parts:
            for specific_part in range(len(part_options[broad_part])):
                for specific_supplier in part_options[broad_part].iloc[specific_part]['Suppliers']:
                    if supplier == specific_supplier:
                        first_row_to_add = [supplier, 
                                            broad_part, 
                                            part_options[broad_part].index[specific_part],
                                            1,
                                            random.randint(5,10),
                                            round(random.uniform(0.1, 0.15), 2),
                                            ]
                        
                        second_row_to_add = [supplier, 
                                            broad_part, 
                                            part_options[broad_part].index[specific_part],
                                            2,
                                            random.randint(15, 20),
                                            round(random.uniform(0.20, 0.30), 2),
                                            ]
                        
                        third_row_to_add = [supplier, 
                                            broad_part, 
                                            part_options[broad_part].index[specific_part],
                                            3,
                                            random.randint(25,40),
                                            round(random.uniform(0.35, 0.50), 2),
                                            ]
                        
                        qds_df.loc[len(qds_df)] = first_row_to_add
                        qds_df.loc[len(qds_df)] = second_row_to_add
                        qds_df.loc[len(qds_df)] = third_row_to_add
                        
    return qds_df

In [15]:
qds = make_quantity_discount_schedule(Suppliers = supplier_list, 
                                      part_options = partOptions, 
                                      Broad_parts = parts_list, 
                                      random_seed = 10
                                     )

print(qds.to_string())

      Supplier      Broader Part  Specific Part: Row Index  Price Level  MinQty  Discount
0    Supplier1             wings                       885            1       9      0.10
1    Supplier1             wings                       885            2      18      0.26
2    Supplier1             wings                       885            3      31      0.42
3    Supplier1             wings                      1354            1       8      0.14
4    Supplier1             wings                      1354            2      20      0.28
5    Supplier1             wings                      1354            3      26      0.43
6    Supplier1             wings                      2591            1       7      0.10
7    Supplier1             wings                      2591            2      20      0.30
8    Supplier1             wings                      2591            3      26      0.41
9    Supplier1             wings                      2532            1       6      0.13
10   Suppl

In [16]:
# Write Quantity Discount Schedule Dataframe to csv file

qds.to_csv(r'/Users/cuhles/OneDrive - University of Tennessee/NSF-SupplyChain_Updated/sae-dfsc/saedfsc/resources/FullQuantityDiscountSchedule.csv')